In [2]:
import numpy as np
from evaldet.det.coco import calculate_pr_curve

In [7]:
preds_bbox = np.array([
    [1, 1, 1, 1],
    [0, 0, 1, 1],
    [1, 1, 1, 1],
    [0, 0, 1, 1],
], dtype=np.float32)
gts_bbox = np.array([
    [1, 1, 1, 1],
    [0, 0, 1, 1],
    [1, 1, 1, 1],
    [0, 0, 1, 1],
], dtype=np.float32)
# gts_bbox=np.zeros((0,4), dtype=np.float32)
preds_conf = np.array([0.9,0.8,0.6,0.7], dtype=np.float32)

ious = {
    0: np.array([[1.0,0.0],[0.0,1.0]], dtype=np.float32),
    1: np.array([[0.0],[1.0]], dtype=np.float32),
}
# ious = {0:np.zeros((2,0), dtype=np.float32),1:np.zeros((2,0), dtype=np.float32)}
fp = {0: (0,2), 1: (2,4)}
fg = {0: (0,2), 1: (2,3)}
# fg={}
area_range = (0.5, 100)
iou_threshold = 0.5

In [8]:
calculate_pr_curve(
    preds_bbox, gts_bbox, ious, preds_conf, fp, fg, area_range, iou_threshold
)

[ 0  1 -1  0] [False False False False] 3


array([[1.        , 1.        , 1.        , 0.75      ],
       [0.33333334, 0.6666667 , 1.        , 1.        ]], dtype=float32)

In [3]:
preds_bbox = np.array([
    [1, 1, 99, 99],
    [1000, 1000, 1, 1],
    [0, 0, 200, 200],
    [100, 100, 10, 10]
])
gts_bbox = np.array([[0, 0, 100, 100], [100, 100, 10, 10]])
preds_conf = np.array([0.9, 0.99, 0.8, 0.1])
area_range = (101, 1000**2)
iou_threshold = 0.3

NameError: name 'np' is not defined

In [38]:
evaluate_image(preds_bbox, gts_bbox, preds_conf, area_range, iou_threshold)

(array([False,  True, False]), array([0.99, 0.9 , 0.8 ]), 1)

In [40]:
evaluate_image(np.zeros((0,4)), np.zeros((0,4)), np.array([]), area_range, iou_threshold)

ValueError: attempt to get argmax of an empty sequence

In [ ]:
# For the per class thingy:
# * do it per class, parallelize within the class on images
# * first, filter detections to contain only objects of the class
# * second, go over images in class, appending results
# * for the results, compute precision-recall curve (no need to interpolate anything)
# * return the two arrays (precision and recall)
# * in a higher-level function compute mAP per class, and also average over classes

In [5]:
from evaldet.det.coco import evaluate_image, iou_dist
# from evaldet.dist import iou_dist_manual
import numpy as np
N=100
pbb = np.random.rand(N, 4).astype(np.float32)
gtbb = np.random.rand(N-1, 4).astype(np.float32)
pc = np.random.rand(N).astype(np.float32)
a = (0, float("inf"))
iout = 0.1

ious = iou_dist(pbb, gtbb)

In [6]:
_ = evaluate_image(pbb,gtbb,ious,pc,a,iout)

In [7]:
%%timeit
evaluate_image(pbb,gtbb,ious,pc,a,iout)

30.9 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
%%timeit
iou_dist(pbb, gtbb)

508 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [5]:
%%timeit
iou_dist_manual(pbb, gtbb)

6.13 ms ± 587 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
import numba

@numba.njit(parallel=False)
def x(y1,y2):
    a = np.zeros((y1.shape[0], y2.shape[0]))
    for i in numba.prange(50):
        a += iou_dist(y1,y2+i)

    return a



In [5]:
x(pbb, gtbb)

array([[50.        , 50.        , 50.        , ..., 50.        ,
        50.        , 50.        ],
       [50.        , 50.        , 50.        , ..., 49.728993  ,
        49.5525378 , 50.        ],
       [49.97867173, 49.84659052, 50.        , ..., 49.63559186,
        49.77126575, 49.99522007],
       ...,
       [49.77200723, 50.        , 49.98689926, ..., 49.95099527,
        50.        , 49.86190456],
       [49.99984944, 49.94537628, 50.        , ..., 49.73895621,
        49.78806257, 50.        ],
       [49.63546646, 50.        , 50.        , ..., 49.9896943 ,
        50.        , 49.67482722]])

In [7]:
%%timeit
x(pbb, gtbb)

55.6 ms ± 3.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
import numba
import numpy as np
# x = {1: np.array([[1,1]], dtype=np.float32)}

d = numba.typed.Dict.empty(
    key_type=numba.types.int32,
    value_type=numba.types.float32[:,::1],
)
d[1] = np.array([[1,1]], dtype=np.float32)
# numba.typed.List(x)

@numba.njit
def xf(y):
    return y

xf(d)

/home/tadej/micromamba/envs/EvalDeT/lib/python3.10/site-packages/numba/typed/typeddict.py:33: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  d[key] = value


DictType[int32,array(float32, 2d, C)]<iv=None>({1: [[1. 1.]]})

In [11]:
xf(d)

DictType[int32,array(float32, 2d, C)]<iv=None>({1: [[1. 1.]]})

In [12]:
numba.typeof(d)

DictType[int32,array(float32, 2d, C)]<iv=None>

In [9]:
import numpy as np

a = np.array([[1,2],[3,4]])

a[[1,0],:][:,[1,0]]

array([[4, 3],
       [2, 1]])

In [10]:
a.T

array([[1, 3],
       [2, 4]])